In [1]:
MODEL_NAME = "intfloat/multilingual-e5-small"

WORKING_DIR = "/Users/hotchpotch/src/huggingface.co/datasets/hotchpotch/wikipedia-passages-jawiki-embeddings/"
# WORKING_DIR = '/home/hotchpotch/src/huggingface.co/datasets/hotchpotch/wikipedia-passages-jawiki-embeddings/'

WIKIPEDIA_DS = "singletongue/wikipedia-utils"
WIKIPEDIA_DS_NAME = "passages-c400-jawiki-20230403"
# DS_NAME = 'hotchpotch/wikipedia-ja-20231030'
INDEX_NAME = "faiss_indexes/passages-c400-jawiki-20230403/multilingual-e5-small-passage/index_m96_mbit8_nlist1024.faiss"

In [2]:
from datasets.download import DownloadManager
from datasets import load_dataset

ds = load_dataset(path=WIKIPEDIA_DS, name=WIKIPEDIA_DS_NAME, split="train")
dm = DownloadManager()
# index_pass  = dm.download(f"https://huggingface.co/datasets/{DS_NAME}/resolve/main/{INDEX_NAME}")

/Users/hotchpotch/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds

Dataset({
    features: ['id', 'pageid', 'revid', 'title', 'section', 'text'],
    num_rows: 5555583
})

In [4]:
import torch
from sentence_transformers import SentenceTransformer


def get_model(model_name: str, max_seq_length=512):
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    model = SentenceTransformer(model_name, device=device)
    model.max_seq_length = max_seq_length
    return model


MODEL = get_model(MODEL_NAME)

In [5]:
WORKING_DIR + INDEX_NAME

'/Users/hotchpotch/src/huggingface.co/datasets/hotchpotch/wikipedia-passages-jawiki-embeddings/faiss_indexes/passages-c400-jawiki-20230403/multilingual-e5-small-passage/index_m96_mbit8_nlist1024.faiss'

In [6]:
import faiss

index = faiss.read_index(WORKING_DIR + INDEX_NAME)
index.ntotal

5555583

In [10]:
index.nprobe = 512

In [11]:
import numpy as np

prefix = ""

if "-e5-" in INDEX_NAME:
    prefix = "query: "


def to_emb(text, prefix=prefix) -> np.ndarray:
    return MODEL.encode([prefix + text], normalize_embeddings=True)  # type: ignore


emb = to_emb("フリーレンの声優は誰?")
emb.shape

(1, 384)

In [12]:
import time


def faiss_search_by_text(text, ds, faiss_index=index, top_k=10):
    start_time = time.time()
    emb = to_emb(text)
    emb_time = time.time()
    print(f"Embedding took {emb_time - start_time:.2f} seconds")
    D, I = faiss_index.search(emb, top_k)
    end_time = time.time()
    print(f"Search took {end_time - emb_time:.2f} seconds")
    results = []
    for idx, (id, d) in enumerate(zip(I[0], D[0])):
        if idx >= 0:
            id = int(id)
            results.append((ds[id], d))
    return results


q = "宝川温泉が東の横綱、湯原温泉が西の横綱と呼ばれる、温泉地ではおなじみの野外に設置されたお風呂を何というでしょう?"
q = "日本で最大の湖は?"
q = "ラーメン食べたい?"
q = "1975年に『アザミ嬢のララバイ』でデビューした女性歌手で、『わかれうた』『地上の星』などの曲を出しているのは誰?"

for data, score in faiss_search_by_text(q, ds):
    print(score, data["title"], data["text"])
    # print(list(data.keys()))

Embedding took 0.02 seconds
Search took 0.07 seconds
0.21803653 アザミ嬢のララバイ 「アザミ嬢のララバイ」(アザミじょうのララバイ)は、中島みゆきのデビュー・シングル。1975年9月25日にキャニオン・レコードよりリリース(レーベルはAARD-VARK)。規格品番:AV-69。
0.24827915 アザミ嬢のララバイ 3⁄4拍子のワルツである。1stアルバム『私の声が聞こえますか』にはアレンジ違いのアルバムバージョンが収録された。本作発表・発売前の1972年に「フォーク音楽祭全国大会」に出場し「あたし時々おもうの」で入賞しており、この時の音源は大会実況版としてLP化、発売されている。そのような経緯もありレコード・デビューとプロ・デビューは年月が異なるが、本格的なデビューとしては本作から起算されている。発売前に、父親が脳溢血で倒れ昏睡状態で病院に運ばれたため、中島本人は宣伝活動ができなかった。オリジナル7インチシングル盤の歌詞カードには中島のプロフィールと、本作を制作した時の心境(「ララバイ」について一言)、及び「アザミ」についての一言が記載されている。編曲家として活動している、船山基紀のデビュー作である。
0.25072595 わかれうた 今作で、自身初のオリコンシングルチャート1位を獲得した。この後中島は、オリコンにおいて4つの年代に渡ってシングルチャート1位を獲得することになる。カップリングの「ホームにて」が、JR東日本の特別企画乗車券(トクトクきっぷ)「ふるさと行きの乗車券」のラジオCMソングに使われている。ヒットしていた当時、フジテレビ『夜のヒットスタジオ』に同曲でテレビ出演。『アザミ嬢のララバイ』と本作を披露した。その際の姿は黒いドレスに裸足でアコースティック・ギターという出で立ちであった。1978年1月19日放送開始のTBSテレビ『ザ・ベストテン』第1回の放映で4位にランクインしたが、同番組への出演は一切拒否していた。ちなみに3位は、桜田淳子に楽曲提供した「しあわせ芝居」だった。
0.2827809 アザミ嬢のララバイ 1978年に「わかれうた」がヒットした際、『夜のヒットスタジオ』に出演。本作と「わかれうた」を披露した。
0.28566822 わかれうた 「わかれうた」は、中島みゆきの楽